# Análise de Dados Educacionais: ENEM 2022
## Notebook 1: Tratamento e Exploração de Dados

Este notebook demonstra o processo de carregamento, limpeza e exploração inicial dos dados do ENEM 2022, Censo Escolar 2022 e indicadores municipais.

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configurações de visualização
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(font_scale=1.2)
pd.set_option('display.max_columns', None)

## 1. Carregamento dos Dados

Vamos carregar os dados brutos do ENEM, Censo Escolar e indicadores municipais.

In [ ]:
# Definir caminho base
BASE_PATH = r"C:\Users\Administrator\Videos\projeto_final_enem"

# Carregar dados do ENEM
enem_df = pd.read_csv(f'{BASE_PATH}\dados\enem_2022_amostra.csv', sep=';')

# Carregar dados do Censo Escolar
censo_escolar_df = pd.read_csv(f'{BASE_PATH}\dados\censo_escolar_2022_amostra.csv', sep=';')

# Carregar dados de indicadores municipais
municipios_df = pd.read_csv(f'{BASE_PATH}\dados\indicadores_municipios.csv', sep=';')

print(f"ENEM: {enem_df.shape[0]} linhas e {enem_df.shape[1]} colunas")
print(f"Censo Escolar: {censo_escolar_df.shape[0]} linhas e {censo_escolar_df.shape[1]} colunas")
print(f"Indicadores Municipais: {municipios_df.shape[0]} linhas e {municipios_df.shape[1]} colunas")

## 2. Exploração Inicial dos Dados

Vamos examinar as primeiras linhas de cada conjunto de dados e obter informações sobre suas estruturas.

In [ ]:
# Explorar dados do ENEM
print("===== Primeiras linhas do ENEM =====")
enem_df.head()

In [ ]:
# Informações sobre as colunas do ENEM
print("===== Informações do ENEM =====")
enem_df.info()

In [ ]:
# Estatísticas descritivas do ENEM
print("===== Estatísticas descritivas do ENEM =====")
enem_df.describe()

In [ ]:
# Explorar dados do Censo Escolar
print("===== Primeiras linhas do Censo Escolar =====")
censo_escolar_df.head()

In [ ]:
# Explorar dados de indicadores municipais
print("===== Primeiras linhas dos Indicadores Municipais =====")
municipios_df.head()

## 3. Verificação de Valores Nulos

Vamos verificar a presença de valores nulos em cada conjunto de dados.

In [ ]:
# Verificar valores nulos no ENEM
print("===== Valores nulos no ENEM =====")
enem_df.isnull().sum()

In [ ]:
# Verificar valores nulos no Censo Escolar
print("===== Valores nulos no Censo Escolar =====")
censo_escolar_df.isnull().sum()

In [ ]:
# Verificar valores nulos nos Indicadores Municipais
print("===== Valores nulos nos Indicadores Municipais =====")
municipios_df.isnull().sum()

## 4. Tratamento dos Dados do ENEM

Vamos realizar o tratamento dos dados do ENEM, incluindo a criação de colunas derivadas.

In [ ]:
# Criar uma cópia para não modificar os dados originais
enem_tratado = enem_df.copy()

# Tratando valores nulos nas notas - substituir por zero os que não fizeram a prova
notas_cols = [col for col in enem_tratado.columns if 'NOTA' in col]
for col in notas_cols:
    enem_tratado[col] = enem_tratado[col].fillna(0)

# Criar coluna de média das notas
enem_tratado['MEDIA_NOTAS'] = enem_tratado[['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT']].mean(axis=1)

# Criar coluna de faixa etária
bins = [0, 17, 20, 25, 30, 100]
labels = ['Até 17 anos', '18 a 20 anos', '21 a 25 anos', '26 a 30 anos', 'Acima de 30 anos']
enem_tratado['FAIXA_ETARIA'] = pd.cut(enem_tratado['NU_IDADE'], bins=bins, labels=labels)

# Mapear tipo de escola para texto
enem_tratado['TIPO_ESCOLA'] = enem_tratado['TP_ESCOLA'].map({1: 'Pública', 2: 'Privada'})

# Verificar as novas colunas
print("===== Dados do ENEM após tratamento =====")
enem_tratado[['NU_INSCRICAO', 'TP_SEXO', 'NU_IDADE', 'FAIXA_ETARIA', 'TIPO_ESCOLA', 'MEDIA_NOTAS']].head()

## 5. Tratamento dos Dados do Censo Escolar

Vamos realizar o tratamento dos dados do Censo Escolar, incluindo a criação de índices de infraestrutura.

In [ ]:
# Criar uma cópia para não modificar os dados originais
censo_tratado = censo_escolar_df.copy()

# Filtrar apenas escolas de Ensino Médio
censo_tratado = censo_tratado[censo_tratado['IN_ENSINO_MEDIO'] == 1]

# Criar coluna com indicador de infraestrutura
infra_cols = ['IN_BIBLIOTECA', 'IN_LABORATORIO_INFORMATICA', 'IN_LABORATORIO_CIENCIAS',
             'IN_QUADRA_ESPORTES', 'IN_SALA_ATENDIMENTO_ESPECIAL', 'IN_INTERNET']
censo_tratado['NIVEL_INFRAESTRUTURA'] = censo_tratado[infra_cols].sum(axis=1)

# Categorizar o nível de infraestrutura
bins_infra = [-1, 2, 4, 6]
labels_infra = ['Básica', 'Intermediária', 'Avançada']
censo_tratado['CATEGORIA_INFRAESTRUTURA'] = pd.cut(censo_tratado['NIVEL_INFRAESTRUTURA'], 
                                                 bins=bins_infra, labels=labels_infra)

# Verificar as novas colunas
print("===== Dados do Censo Escolar após tratamento =====")
censo_tratado[['CO_ENTIDADE', 'NO_ENTIDADE', 'NIVEL_INFRAESTRUTURA', 'CATEGORIA_INFRAESTRUTURA']].head()

## 6. Tratamento dos Dados de Indicadores Municipais

Vamos realizar o tratamento dos dados de indicadores municipais, incluindo a categorização do IDH.

In [ ]:
# Criar uma cópia para não modificar os dados originais
municipios_tratado = municipios_df.copy()

# Criar coluna de categorização do IDH
bins_idh = [0, 0.5, 0.6, 0.7, 0.8, 1.0]
labels_idh = ['Muito baixo', 'Baixo', 'Médio', 'Alto', 'Muito alto']
municipios_tratado['CATEGORIA_IDH'] = pd.cut(municipios_tratado['IDH'], bins=bins_idh, labels=labels_idh)

# Verificar as novas colunas
print("===== Dados dos Indicadores Municipais após tratamento =====")
municipios_tratado[['CODIGO_IBGE', 'NOME_MUNICIPIO', 'IDH', 'CATEGORIA_IDH']].head()

## 7. Mesclando os Dados

Vamos mesclar os três conjuntos de dados para análises integradas.

In [ ]:
# Mesclar ENEM com dados municipais
print("===== Mesclando ENEM com Indicadores Municipais =====")
enem_municipios = pd.merge(
    enem_tratado, 
    municipios_tratado,
    left_on='CO_MUNICIPIO_RESIDENCIA',
    right_on='CODIGO_IBGE',
    how='left'
)

print(f"Resultado: {enem_municipios.shape[0]} linhas e {enem_municipios.shape[1]} colunas")

# Mesclar com dados do Censo Escolar
print("\n===== Mesclando com Censo Escolar =====")
dados_completos = pd.merge(
    enem_municipios,
    censo_tratado,
    left_on='CO_ESCOLA',
    right_on='CO_ENTIDADE',
    how='left'
)

print(f"Resultado final: {dados_completos.shape[0]} linhas e {dados_completos.shape[1]} colunas")

# Verificar as primeiras linhas do conjunto de dados mesclado
dados_completos.head()

## 8. Salvando os Dados Tratados

Vamos salvar os conjuntos de dados tratados para uso posterior.

In [ ]:
# Salvar dados tratados
enem_tratado.to_csv(f'{BASE_PATH}\dados_tratados\enem_tratado.csv', index=False)
censo_tratado.to_csv(f'{BASE_PATH}\dados_tratados\censo_escolar_tratado.csv', index=False)
municipios_tratado.to_csv(f'{BASE_PATH}\dados_tratados\municipios_tratado.csv', index=False)
dados_completos.to_csv(f'{BASE_PATH}\dados_tratados\dados_completos.csv', index=False)

print("Todos os dados tratados foram salvos com sucesso!")

## 9. Resumo e Próximos Passos

Neste notebook, realizamos o carregamento, exploração e tratamento dos dados do ENEM 2022, Censo Escolar 2022 e indicadores municipais. Criamos diversas colunas derivadas, incluindo média das notas, faixa etária, nível de infraestrutura e categoria de IDH. Também mesclamos os dados para análises integradas.

No próximo notebook, realizaremos análises mais aprofundadas e criaremos visualizações para identificar padrões e tendências nos dados.